# **Libraries**

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Bidirectional

# **Importing Dataset**

In [4]:
url = 'https://github.com/haysacks/midi-generator/raw/master/jazz-midi.csv'
df = pd.read_csv(url, index_col = 0)
print(df.shape)
df.head()

(818, 5)


,Name,Notes,Len_Sequence,Unique_notes,len_Uni_Notes
0,BreezeAndI.mid,"['E-2', 'B-2', 'B-2', 'E-3', 'B-2', 'B-2', 'E-...",427,"{'C3', 'F1', 'A2', 'B-0', 'E-1', 'E3', 'B1', '...",27
1,IfIHadYou.mid,"['10.2.5', '10.2.5', '2.5.8', '2.5.8', '0.3.7'...",204,"{'7.10.2', '6.9.0.2', '9.10.0.2.4', '6.9.0', '...",37
2,IllBeSeeingYou.mid,"['E-2', 'E-2', 'B-2', 'B-2', 'G2', 'D3', 'F2',...",455,"{'C3', 'F1', 'B-0', 'A2', 'E-1', 'D3', 'B1', '...",29
3,JustAGame.mid,"['B3', '6.11', 'B1', 'E4', 'B1', 'F#4', 'B1', ...",1572,"{'C3', '9.2', 'A2', 'B4', 'E3', 'A3', 'D5', 'B...",39
4,Unforgettable.mid,"['G2', 'G1', 'G2', 'G1', 'G2', 'G2', 'G2', 'C#...",421,"{'A0', 'C3', 'F1', 'A2', 'E-1', 'E3', 'G1', 'D...",28


We take the first 100 MIDI files from the dataset due to time constraints.

In [5]:
# Combine all notes into 1 list
notes = df["Notes"][0:100].str.replace("'", "").str.replace(' ', '').str.strip('[]').str.split(',')
notes = [j for i in list(notes) for j in i]

with open('notes.pickle', 'wb') as filename:
    pickle.dump(notes, filename)
print("Saved notes list to disk")

Saved notes list to disk


**Making sequences and recording the following note/chord**

In [6]:
pitchnames = sorted(set(item for item in notes))
# Map notes/chords into integer categories
notes_categories = dict((note, number) for number, note in enumerate(pitchnames))

# Predict 1 note for sequence of <sequence_length> notes
sequence_length = 50
length = len(notes) - sequence_length

# Input and output for LSTM
x = []
y = []

for i in range(length):
    sequence_in = notes[i : i + sequence_length]
    sequence_out = notes[i + sequence_length]
    
    x.append([notes_categories[note] for note in sequence_in])
    y.append(notes_categories[sequence_out])

with open('x.pickle', 'wb') as filename:
  pickle.dump(x, filename)
print("Saved x to disk")

Saved x to disk


# **Data Preparation**

Reshape and normalise input, and one-hot encode categorical output

In [7]:
n_patterns = len(x)
# Number of unique notes
n_vocab = len(set(notes))

print(n_patterns)
print(n_vocab)

# Input of LSTM layer is 3D (samples x time steps x features)
x = np.reshape(x, (n_patterns, sequence_length, 1))

# Normalise input
x = x / float(n_vocab)

# Encode categorical output
y = np.array(y)
ohe = OneHotEncoder(sparse = False)
y = y.reshape(len(y), 1)
y = ohe.fit_transform(y)
print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

85070
515
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# **LSTM Model**

In [8]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(x_train.shape[1], x_train.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256))
model.add(Dropout(0.5))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 512)           1052672   
_________________________________________________________________
dropout (Dropout)            (None, 50, 512)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 1024)          4198400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

**Training model**

In [9]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50, batch_size = 72)

Train on 68056 samples, validate on 17014 samples
Epoch 1/50
 3888/68056 [>.............................] - ETA: 31:45 - loss: 5.1474 - accuracy: 0.0320

KeyboardInterrupt: 

**Saving model and weights**

In [ ]:
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")